In [ ]:
import data_exploration
from combined_attributes_adder import CombinedAttributesAdder
from feature_selector_transformer import FeatureSelectorTransformer

import os

import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, \
    GridSearchCV, RandomizedSearchCV

from sklearn.svm import SVR

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy import stats


In [ ]:
HOUSING_PATH = os.path.join("datasets", "housing")

housing = data_exploration.load_housing_data(HOUSING_PATH)
housing.head()

In [ ]:
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
housing.hist(bins=50, figsize=(20, 15))

## Split the dataset into train and test subsets

### Example using simple random splitting

In [ ]:
# train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

### Example using stratified sampling

In [ ]:
feature_for_splitting = 'median_income'
bin_edges = [0., 1.5, 3.0, 4.5, 6., np.inf]
test_fraction_of_dataset = 0.2
seed = 42

train_set, test_set = data_exploration.split_into_training_test_datasets(housing, 
                                                                         feature_for_splitting, 
                                                                         bin_edges, 
                                                                         test_fraction_of_dataset, 
                                                                         seed)

In [ ]:
train_set.head()

## Explore the data

In [ ]:
housing = train_set.copy()

In [ ]:
housing.plot(kind='scatter', 
             x='longitude', 
             y='latitude', 
             alpha=0.4, 
             s=housing['population']/100,
             label='population',
             figsize=(10,7),
             c=housing['median_house_value'],
             cmap=plt.get_cmap('jet'),
             colorbar=True)
plt.legend()

## Look for correlations

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)

## Create combinations of feature

In [ ]:
housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
housing['population_per_household'] = housing['population'] / housing['households']

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

## Data cleaning

In [ ]:
train_set.head()

In [ ]:
housing = train_set.drop('median_house_value', axis=1).copy()  # remove the column used for the labels

In [ ]:
housing_labels = train_set['median_house_value'].copy()

### Replace missing values with the median of that feature

In [ ]:
imputer = SimpleImputer(strategy='median')
housing_num = housing.drop('ocean_proximity', axis=1)  # Can only calculate median on numerical values

## Transformation pipeline
### Create a pipeline for the transformation of numerical features
* SimpleImputer replaces all null values with the median for that feature
* CombinedAttributesAdder is a bespoke class that combines attributes
* StandardScaler subtracts the mean and divides by the standard deviation

### Saving training data to file

In [ ]:
TRAINING_DATA_SAVE = './data/training_data'

housing_num.to_pickle(TRAINING_DATA_SAVE)
df = pd.read_pickle(TRAINING_DATA_SAVE)
df

In [ ]:
features_to_use = [True, True, True, False, False, True, True, True, True, True, True]

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
    ('feature_choice', FeatureSelectorTransformer(columns_to_use = features_to_use, pandas=False))
])

#housing_num_tr = num_pipeline.fit_transform(housing_num)

### Create a pipeline for the transformation of text features and combine with numerical pipeline

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

### Create a pipeline including the regression

In [ ]:
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinearRegression())
    ])

full_pipeline_with_predictor.fit(housing, housing_labels)
full_pipeline_with_predictor.predict(some_data)

## Train a linear regression model

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

### Evaluate model

In [ ]:
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

## Train a decision tree regressor

In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

### Evaluation model

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

## Train a random forest regressor

In [ ]:
forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

## Train a support vector regressor:

In [ ]:
support_vector_regressor = SVR(kernel='rbf', C=1, gamma='scale')
support_vector_regressor.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = support_vector_regressor.predict(housing_prepared)
svr_mse = mean_squared_error(housing_labels, housing_predictions)
svr_rmse = np.sqrt(svr_mse)
svr_rmse

## Use cross-validation
### Decision tree:

In [ ]:
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print('scores', scores)
    print('mean', scores.mean())
    print('sd', scores.std())
    
display_scores(tree_rmse_scores)

### Linear model:

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring='neg_mean_squared_error', cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

display_scores(lin_rmse_scores)

### Random forrest:

In [ ]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

## Hyper-parameter search
### Grid search:

In [ ]:
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, 
                           param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print('{0:.0f}\t{1:}'.format(np.sqrt(-mean_score), params))

### Randomised search:

In [ ]:
param_dists = {'n_estimators': stats.randint(low=29, high=30)}

forest_reg = RandomForestRegressor()

random_search = RandomizedSearchCV(forest_reg, 
                                 param_distributions = param_dists,
                                 n_iter=1,
                                 cv=5, 
                                 scoring='neg_mean_squared_error', 
                                 return_train_score=True)

random_search.fit(housing_prepared, housing_labels)

In [ ]:
random_search.best_params_

In [ ]:
random_search.best_estimator_

In [ ]:
cvres = random_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print('{0:.0f}\t{1:}'.format(np.sqrt(-mean_score), params))

In [ ]:
#feature_importances = random_search.best_estimator_.feature_importances_

extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
attributes
#sorted(zip(feature_importances, attributes), reverse=True)

# Evaluation on the test dataset
### Get the final model parameters & prepare the test data into input and labels

In [ ]:
final_model = grid_search.best_estimator_

In [ ]:
X_test = test_set.drop('median_house_value', axis=1)
y_test = test_set['median_house_value'].copy()

### Transform the test data in the same way as the training data 

In [ ]:
X_test_prepared = full_pipeline.transform(X_test)

### Make predictions & evaluation error

In [ ]:
final_predictions = final_model.predict(X_test_prepared)

In [ ]:
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

In [ ]:
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence,
                         len(squared_errors)-1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))